In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Bidirectional

# Step 1: Load the data from the CSV file
data_path = '/Users/charansaisadla/Desktop/ub classes/SEM2/NLP/data project/'
data=pd.read_csv(data_path+"input.csv")


texts = data['text'].tolist()
classes = data['class'].tolist()

# Step 2: Convert the text data into numerical representations
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

tokenized_texts = []
for text in texts:
    tokens = [lemmatizer.lemmatize(token.lower()) for token in word_tokenize(text) if
              token.lower() not in stop_words and token.lower() not in string.punctuation]
    tokenized_texts.append(tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(tokenized_texts)
max_sequence_length = 500
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Step 3: Encode the class labels
label_encoder = LabelEncoder()
encoded_classes = label_encoder.fit_transform(classes)
num_classes = len(label_encoder.classes_)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_classes, test_size=0.2, random_state=42)

# Step 5: Define and train the CRNN model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(Conv1D(256, 8, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)

# Step 6: Evaluate the model on the test set
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)
report = classification_report(y_test, y_pred)
print(report)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/5
37/37 [==============================] - 29s 777ms/step - loss: 1.8346 - accuracy: 0.3382 - val_loss: 1.1481 - val_accuracy: 0.5286
Epoch 2/5
37/37 [==============================] - 32s 867ms/step - loss: 0.8009 - accuracy: 0.7061 - val_loss: 0.7047 - val_accuracy: 0.7489
Epoch 3/5
37/37 [==============================] - 38s 1s/step - loss: 0.3235 - accuracy: 0.8966 - val_loss: 0.5959 - val_accuracy: 0.7992
Epoch 4/5
37/37 [==============================] - 41s 1s/step - loss: 0.1094 - accuracy: 0.9704 - val_loss: 0.6111 - val_accuracy: 0.8167
Epoch 5/5
74/74 [==============================] - 4s 54ms/step
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       214
           1       1.00      1.00      1.00       217
           2       0.82      0.50      0.62       213
           3       0.79      0.90      0.84       431
           4       0.99      0.98      0.98       215
           5       0.97      0.97      0.97      